In [8]:
import os
import numpy as np
import pandas as pd

## Import data

In [9]:
# load doc into memory
def load_doc(filename):
    # open the file as read only 
    file = open(filename, 'r')
    # read all text
    text = file.read()
    text = text.splitlines()
    # close the file file.close()
    return text

In [10]:
dir_path = os.path.realpath('..')

In [11]:
path = 'data/raw/train.csv'

full_path = os.path.join(dir_path, path)
df_train = pd.read_csv(full_path, header=0, index_col=0)
print("Dataset has {} rows, {} columns.".format(*df_train.shape))

Dataset has 95851 rows, 7 columns.


In [12]:
path = 'data/raw/test.csv'

dir_path = os.path.realpath('..')
full_path = os.path.join(dir_path, path)
df_test = pd.read_csv(full_path, header=0, index_col=0)
print("Dataset has {} rows, {} columns.".format(*df_test.shape))

Dataset has 226998 rows, 1 columns.


In [13]:
path = 'data/processed/vocab.txt'

dir_path = os.path.realpath('..')
full_path = os.path.join(dir_path, path)
vocab = load_doc(full_path)

In [14]:
# fill NaN with string "unknown"
df_train.fillna('unknown',inplace=True)
df_test.fillna('unknown',inplace=True)

## Glove

In [15]:
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [16]:
%%time
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(vocab)
vocab_size = len(t.word_index) + 1

CPU times: user 242 ms, sys: 7.94 ms, total: 250 ms
Wall time: 248 ms


In [17]:
%%time
# integer encode the documents
Xtrain = t.texts_to_sequences(df_train.comment_text)
Xtest = t.texts_to_sequences(df_test.comment_text)

CPU times: user 28.8 s, sys: 238 ms, total: 29 s
Wall time: 29.3 s


In [18]:
# pad documents to a max length of 4 words
max_length = 4
padded_train = pad_sequences(Xtrain, maxlen=max_length, padding='post')
padded_test = pad_sequences(Xtest, maxlen=max_length, padding='post')

In [19]:
%%time
# load the whole embedding into memory
embeddings_index = dict()
f = open('/Users/joaeechew/dev/glove.6B/glove.6B.100d.txt', mode='rt', encoding='utf-8')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.
CPU times: user 11.6 s, sys: 524 ms, total: 12.2 s
Wall time: 12.6 s


In [20]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [23]:
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [13]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 100)            2428000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 401       
Total params: 2,428,401
Trainable params: 401
Non-trainable params: 2,428,000
_________________________________________________________________


In [14]:
def save_model(model, model_name):
    # serialize model to JSON
    model_json = model.to_json()
    with open(model_name + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(model_name + ".h5")
    print("Saved model to disk")

In [20]:
%%time
# fit the model
target = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

for label in target:
    print('... Processing {}'.format(label))
    ytrain = df_train[label]
    model_name = 'glove_model_' + label
    
    # train the model
    model.fit(padded_docs, ytrain, epochs=1, verbose=2)
    
    # save the model
    save_model(model, model_name)

... Processing toxic
Epoch 1/1
 - 6s - loss: 0.2486 - acc: 0.9175
Saved model to disk
... Processing severe_toxic
Epoch 1/1
 - 6s - loss: 0.0463 - acc: 0.9884
Saved model to disk
... Processing obscene
Epoch 1/1
 - 6s - loss: 0.1483 - acc: 0.9568
Saved model to disk
... Processing threat
Epoch 1/1
 - 6s - loss: 0.0221 - acc: 0.9960
Saved model to disk
... Processing insult
Epoch 1/1
 - 6s - loss: 0.1422 - acc: 0.9568
Saved model to disk
... Processing identity_hate
Epoch 1/1
 - 6s - loss: 0.0414 - acc: 0.9909
Saved model to disk
CPU times: user 38.4 s, sys: 18.1 s, total: 56.5 s
Wall time: 35.2 s


## Prediction

In [28]:
from keras.models import model_from_json

In [29]:
def load_model(model_name):
    # load json and create model
    json_file = open(model_name+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(model_name+".h5")
    print("Loaded model from disk")
    return loaded_model

In [52]:
%%time
target = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
submission = pd.DataFrame(index=df_test.index, columns=target)

for label in target:
    print('... Processing {}'.format(label))
    model_name = 'glove_model_' + label

    # load the model
    loaded_model = load_model(model_name)
    
    y_pred_proba = loaded_model.predict(padded_test, verbose=0, batch_size=1)
    submission[label] = y_pred_proba.flatten()

... Processing toxic
Loaded model from disk
... Processing severe_toxic
Loaded model from disk
... Processing obscene
Loaded model from disk
... Processing threat
Loaded model from disk
... Processing insult
Loaded model from disk
... Processing identity_hate
Loaded model from disk
CPU times: user 11min 42s, sys: 1min 24s, total: 13min 6s
Wall time: 9min 26s


In [53]:
path = 'data/processed/submission.csv'

dir_path = os.path.realpath('..')
full_path = os.path.join(dir_path, path)

submission.to_csv(full_path, header=True, index=True)

## Evaluation

In [16]:
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

NameError: name 'labels' is not defined